#### Chapter 7: Model training Philosophies

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,MaxPooling2D,Dropout,Normalization
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

In [ ]:
# load dataset
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

In [ ]:
# Defining the model
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['acc'])
model.summary()

In [ ]:
model.fit(trainX,trainy,epochs=20,validation_split=0.1)

##### Fitting model after normalisation

In [ ]:
# Normalising the train and test sets
trainX_norm = trainX/255
testX_norm = testX/255
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX_norm.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX_norm.shape, testy.shape))

In [ ]:
model.fit(trainX_norm,trainy,epochs=20,validation_split=0.1)

#### Normalising using ImageDataGenerator

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Defining an image data generator
datagenerator = ImageDataGenerator(rescale=1./255,)
# Defining train generator
train_iterator = datagenerator.flow(trainX, trainy, batch_size=64)
# Defining validation generator
val_iterator = datagenerator.flow(testX, testy, batch_size=64)

In [ ]:
# Defining the model
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['acc'])

In [ ]:
model.fit(train_iterator,epochs=20,validation_data=val_iterator)

#### ImageDataGenerator with Z-score normalization

In [ ]:
# ImageDataGenerator with Z-score normalization
mean = 0.5
Std = 0.5
datagenerator = ImageDataGenerator(rescale= (mean/255), (std/255))


In [ ]:
# ImageDataGenerator with log-normalization
import numpy as np
def log_normalise(x):
    return np.log(x+1)
datagenerator = ImageDataGenerator(preprocessing_function = log_normalise)


##### Data Augmentation with ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
from numpy import expand_dims
import matplotlib as mpl
import numpy as np

You need to download any image and give its path instead of 'image2.jpeg'

In [ ]:
# load the image with the required shape
img_input = load_img('image2.jpeg', target_size=(224, 224))
# convert the image to an array
img = img_to_array(img_input)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
print('Image shape',img.shape)
# Displaying the image
img_input

In [ ]:
# Defining an image data generator
datagenerator = ImageDataGenerator(vertical_flip=True,horizontal_flip=True)
# Creating the data iterator
dataiterator = datagenerator.flow(img,batch_size=1)
for i in range(4):
    image = next(dataiterator)[0].astype('uint8')
    plt.figure()
    plt.imshow(image)

##### Shifting

In [ ]:
# Defining an image data generator
datagenerator = ImageDataGenerator(width_shift_range=30,height_shift_range=20)
# Creating the data iterator
dataiterator = datagenerator.flow(img,batch_size=1)
for i in range(4):
    image = next(dataiterator)[0].astype('uint8')
    plt.figure()
    plt.imshow(image)

#### Rotation

In [ ]:
for angle in [15,30,45,60,110]:
    # Defining an image data generator
    datagenerator = ImageDataGenerator(rotation_range = angle)
    # Creating the data iterator
    dataiterator = datagenerator.flow(img,batch_size=1)
    for i in range(1):
        image = next(dataiterator)[0].astype('uint8')
        plt.figure()
        plt.imshow(image)

#### Brightness

In [ ]:
# Defining an image data generator
datagenerator = ImageDataGenerator(brightness_range = (0.3,2))
# Creating the data iterator
dataiterator = datagenerator.flow(img,batch_size=1)
for i in range(5):
    image = next(dataiterator)[0].astype('uint8')
    plt.figure()
    plt.imshow(image)

#### Zoom

In [ ]:
# Defining an image data generator
datagenerator = ImageDataGenerator(zoom_range = (0.3,2))
# Creating the data iterator
dataiterator = datagenerator.flow(img,batch_size=1)
for i in range(4):
    image = next(dataiterator)[0].astype('uint8')
    plt.figure()
    plt.imshow(image)

#### Monitoring using Tensorboard

In [ ]:
!mkdir '/logs/fit'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define the model
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

In [ ]:
!tensorboard --logdir 'logs/fit'

#### Learning rate scheduling

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define the model
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create a callback to log the learning rate during training
lr_callback = keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * (0.1 ** (epoch // 20)))

# Create a TensorBoard callback to log the learning rate, accuracy, and loss
log_dir = 'logs/fit'
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test),
                    callbacks=[lr_callback, tensorboard_callback])

#### Model baselining 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
# Define the model
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Define the early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

# Define the model checkpointing callback
checkpoint_path = "model_checkpoint.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_best_only=True, save_weights_only=False,
    monitor="val_loss", mode="min", verbose=1
)

# Train the model with early stopping and checkpointing
history = model.fit(x_train, y_train, batch_size=32, epochs=25,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping, model_checkpoint])

# Save the model
model.save("model.h5")

# Restore the model from a checkpoint
loaded_model = keras.models.load_model("model_checkpoint.h5")
